# Linear Equations 
In this exercise we will look at solving linear equations. This is a frequqntly encountered task in engineering and science. For example we need to solve a linear equation to find the root of a function using the Newton method. We will look closer at two methods for solving linear equations: Gaussian elimination and LU decomposition.

If you are further interested, you can take a look here https://tobydriscoll.net/fnc-julia/linsys/overview.html. 

## Linear Algebra in Julia
First we need to learn how to do linear algebra in Julia. In the first exercise we learned how to generate and manipulate vectors and matrices. We will now learn how to do linear algebra with these objects. 

## Transposition
As in other languages `A'` is the conjugate transpose, or adjoint

In [ ]:
A = [1 2 3; 4 5 6; 7 8 10]

In [ ]:
A'

In [ ]:
transpose(A)

### Transposed multiplication
Julia allows us to write this without *

In [ ]:
A'A

Note that you can also use `A\b` to solve the linear equation even if `A` is not square. `A\b` gives us the *least squares solution* if we have an overdetermined linear system (a "tall" matrix). If we have an underdetermined linear system (a "short" matrix) `A\b` gives us the minimum norm solution. 

## Linear algebra package
Julia has a built in package for linear algebra called `LinearAlgebra`. This package contains a lot of useful functions for linear algebra. Let's look at some of them.

In [ ]:
using LinearAlgebra

Computing the dot product of two vectors is done using the `dot` function:

In [ ]:
v = [1, 2, 3]
dot(v, v)

The outer product of two vectors is computed using the transpose of the second vector:

In [ ]:
v*v'

We can compute inverse of a matrix using the `inv` function:

In [ ]:
inv(A)

We can use the inverse to solve a linear equation. However, this is not recommended since it is slower and less accurate than using `\`. 

In [ ]:
x = inv(A)*b

### Norm of a vector
There are different commonly used norms of a vector.
- The 2-norm is the Euclidean norm $\|x\|_2 = \sqrt{\sum_{i=1}^n x_i^2}$
- The 1-norm is the sum of the absolute values of the elements $\|x\|_1 = \sum_{i=1}^n |x_i|$
- The $\infty$-norm is the maximum absolute value of the elements $\|x\|_\infty = \max_i |x_i|$

The norm of a vector is computed using the `norm` function:

In [ ]:
x = [2,-3,1,-1]
twonorm = norm(x) # 2-norm

In [ ]:
infnorm = norm(x,Inf)

In [ ]:
onenorm = norm(x,1)

We can also normalize a vector using the `normalize` function:

In [ ]:
normalize(x,2)

### Matrix norms
There are also different matrix norms. If we view the matrix similar to a vector and use the 2-norm we get the Frobenius norm $\|A\|_F = \sqrt{\sum_{i=1}^m \sum_{j=1}^n |a_{ij}|^2}$. 
However, often we want to use the induced matrix norm which is defined as $\|A\| = \max_{x \neq 0} \frac{\|Ax\|}{\|x\|}$. 


In [ ]:
A = [ 2 0; 1 -1 ]

In [ ]:
Fronorm = norm(A)

In [ ]:
twonorm = opnorm(A)

In [ ]:
onenorm = opnorm(A,1) # 1-norm: sum down the first matrix dimension 

In [ ]:
infnorm = opnorm(A,Inf) # infinity-norm: sum down the second matrix dimension

## Conditioning of linear equations
In the lecture we learned that the some linear equations are ill-conditioned or not uniquely solvable. Ill-conditioned means that small changes in the right hand side vector $b$ can lead to large changes in the solution vector $x$.
We can check the determinant of the matrix $A$ to see if the linear equation is ill-conditioned. If the determinant is zero the matrix is singular and the linear equation is not uniquely solvable. Let's check the determinant of the matrix $A$:

In [ ]:
using LinearAlgebra
det(A)

In [ ]:
A_ill = [1 10^-16; 2 10^-16]
det(A_ill)

In [ ]:
b = [1, 2]
x = A_ill\b

In [ ]:
A_singular = [1 2 3; 4 5 6; 7 8 9]
det(A)

In [ ]:
b = [1, 2, 3]
x = A_singular\b

# Gaussian Elimination
In the lecture we learned that we can solve a linear equation using the Gaussian elimination method. The idea is to transform the linear equation into an upper (or lower) triangular matrix. We did this by transforming each row of the matrix. We want to solve for $x$ in the following equation:

$$
\mathbf{A} * \mathbf{x} = \mathbf{b} 
$$
As an example let's look at the following equation:
$$
\begin{pmatrix}
1 & 2 \\
4 & 5 \\
\end{pmatrix}
* x =
\begin{pmatrix}
3 \\
6 \\
\end{pmatrix}
$$
We can rewrite the matrix by extending it with the vector $b$ into $[A b]$:
$$
\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
\end{pmatrix}
$$
We can now perform row operations to transform the matrix into an upper triangular matrix. The first step is to subtract the first row multiplied by $4/1$ from the second row. This will result in the following matrix:
$$
\begin{pmatrix}
1 & 2 & 3 \\
0 & -3 & -6 \\
\end{pmatrix}
$$
We can now solve the second equation for $x_2$ and substitute it into the first equation. This will result in the following equation:
$$
\begin{pmatrix}
1 & 2 & 3 \\
0 & 1 & 2 \\
\end{pmatrix}
$$
We can now solve the first equation for $x_1$ and get the solution $x_1=1$ and $x_2=2$. 


As a more general way to show our example of a 2x2 matrix $A$ we have:
$$
\begin{pmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22} \\
\end{pmatrix}
* x =
\begin{pmatrix}
b_1 \\
b_2 \\
\end{pmatrix}
$$
To perform the Gaussian elimination of this linear equation we can use the following algorithm:
- $\alpha = a_{21}/a_{11}$
- $a_{22} = a_{22} - \alpha * a_{12}$ 
- $b_2 = b_2 - \alpha * b_1$
- $x_2 = b_2/a_{22}$
- $b_1 = b_1 - a_{12} * x_2$
- $x_1 = b_1/a_{11}$
- return $x_1$ and $x_2$

In [ ]:
A = [1 2; 3 4]
b = [5; 6]
A_ = float(copy(A)) # we copy A since changing an element of A will change A as well
b_ = float(copy(b))
α = A_[2,1] / A_[1,1]
A_[2,:] = A_[2,:] - α * A_[1,:]
b_[2] = b_[2] - α * b_[1]
x_2 = b_[2] / A_[2,2]
x_1 = (b_[1] - A_[1,2] * x_2) / A_[1,1]
x = [x_1; x_2]

We can test our implementation by estimating the error of the solution: 
$$
Error = \mathbf{A} * \mathbf{x} - \mathbf{b}
$$  
The error should be zero.

In [ ]:
A*x - b

Can you think of a way to generalize this algorithm to a square matrix of arbitrary size? One difference is that we have to perform the algorithm for each row. Each row will have a different $\alpha$ value. We can use a loop to perform the algorithm for each row. In the example above we have applied the subtraction of the first row multiplied by $4/1$ from the second row. We can generalize this by subtracting the first row multiplied by $a_{21}/a_{11}$ from the second row. Also note that we can actually substract the whole row at once. 

The task is to write a function that implements the Gaussian elimination method for arbitrary square matrices. The function should take the matrix $A$ and the vector $b$ as input and return the solution vector $x$. We will divide this task into two parts. First we will write a function that transforms the linear equation into an upper triangular matrix. Then we will write a function that performs the back substitution to get the solution vector $x$.

Let's start by defining a new matrix $A$ and vector $b$:

In [ ]:
A = [1 2 3; 4 5 6; 7 8 10]
b = [1, 2, 3]

Now we can write a function that transforms the matrix $A$ into an upper triangular matrix. The function should take the matrix $A$ and the vector $b$ as input and return the transformed matrix $A$ and the transformed vector $b$. 

In [ ]:
function gauss_elimination(A::Matrix{T}, b::Vector{T}) where T <: Number
    n = size(A, 1)
    U = float(copy(A))
    b_ = float(copy(b))
    for k = 1:n-1
        for i = k+1:n
            factor = U[i,k] / U[k,k]
            U[i,:] = U[i, :] - (factor * U[k,:])
            b_[i] = b_[i] - (factor * b_[k])
        end
    end
    return U, b_
end

Let's test the function with the matrix $A$ and the vector $b$ defined above. We should get an upper triangular matrix. 

In [ ]:
U, b_ = gauss_elimination(A, b)

Now we can write a function that performs the back substitution to get the solution vector $x$. The function should take the matrix $U$ and the transformed vector $\hat{b}$ as input and return the solution vector $x$.

In [ ]:
function backsub(U,b)
    n = size(U,1)
    x = zeros(n)
    for i = n:-1:1
        x[i] = c[i]
        for j = i+1:n
            x[i] -= U[i,j] * x[j]
        end
        x[i] /= U[i,i]
    end
    return x
end

We can now test the function with the matrix $U$ and the transformed vector $\hat{b}$ defined above. We should get the solution vector $x$.

In [ ]:
x = backsub(U,b)

In [ ]:
A*x-b

## LU Factorization
We learned that we can transform the linear equation with the square matrix $A$ into a upper triangular matrix $U$. In the lecture we learned that we can also transform the linear equation into an lower triangular matrix $L$ and an upper triangular matrix $U$ such that 
$$
A = LU
$$ 
This is called the LU factorization of $A$. The LU factorization is useful for solving linear equations especially when the right hand side vector $b$ is changed. In this case, we can solve the linear equations by solving $Ly = b$ and $Ux = y$. In the example above we would have to perform the whole calculation again. However, if we have the LU factorization of $A$ we can solve the linear equations much faster. 

In the lecture we learned that we can use outer products using the lower triangular matrix $L$ divided into a part for each column. For a 3x3 matrix 
$$
A = 
\begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{pmatrix}
$$
we divide the lower triangular matrix $L$ into two parts:
$$
L = L_1 * L_2 =
\begin{pmatrix}
1 & 0 & 0 \\
a_{21}/a_{11} & 1 & 0 \\
a_{31}/a_{11} & 0 & 1 \\
\end{pmatrix}
*
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & a_{32}/a_{22} & 1 \\
\end{pmatrix}
$$

Using this we can write: 
$$
L_1 * L_2 * A = U
$$

For a general square matrix $A$ we can write:
$$
L_1 * L_2 * l_k * ... * L_{K} * A = U
$$
Where $L_k$ is the lower triangular matrix for the $k$-th column as is has only non-zero values on the diagonal and the k-th column. The $k$-th column of $L_k$ is defined as:
$$
(L_k)_{ik} = \frac{a^{k-1}_{ik}}{a^{k-1}_{kk}}
$$ 
Where $a^{k-1}_{ik}$ is the $i$-th element of the $k$-th column of $A^{k-1} = L_1 * L_2 * l_k * ... * L_{k-1} * A$.


We can now write a function that calculates the LU factorization of a square matrix $A$. The function should take the matrix $A$ as input and return the lower triangular matrix $L$ and the upper triangular matrix $U$.

In [ ]:
using LinearAlgebra
function lufact(A)
    n = size(A,1)
    L = diagm(ones(n))   # ones on main diagonal, zeros elsewhere
    U = zeros(n,n)
    Aₖ = float(copy(A))  # copy of A, converted to float

    # Reduction by outer products
    for k in 1:n-1
        U[k,:] = Aₖ[k,:]
        L[:,k] = Aₖ[:,k]/U[k,k]
        Aₖ -= L[:,k]*U[k,:]'
    end

    U[n,n] = Aₖ[n,n]
    return LowerTriangular(L),UpperTriangular(U)
end

Let's test the function with the matrix $A$ defined above. We should get the lower triangular matrix $L$ and the upper triangular matrix $U$.

In [ ]:
L, U = lufact(A)

Now to solve the linear equation we can use the LU factorization of $A$ and solve the linear equations by solving $Ly = b$ and $Ux = y$. First we define a function for the forward substitution to solve $Ly = b$. The function should take the lower triangular matrix $L$ and the vector $b$ as input and return the solution vector $y$.

In [ ]:
function forwardsub(L,b)
    n = size(L,1)
    y = zeros(n)
    y[1] = b[1]/L[1,1]
    for i in 2:n
        s = sum( L[i,j]*y[j] for j in 1:i-1 )
        y[i] = ( b[i] - s ) / L[i,i]
    end
    return y
end

Now let's define a function for the back substitution to solve $Ux = y$. The function should take the upper triangular matrix $U$ and the vector $y$ as input and return the solution vector $x$.

In [ ]:
function backsub(U,y)
    n = size(U,1)
    x = zeros(n)
    x[n] = y[n]/U[n,n]
    for i in n-1:-1:1
        s = sum( U[i,j]*x[j] for j in i+1:n )
        x[i] = ( y[i] - s ) / U[i,i]
    end
    return x
end

Finally we want to test our implementation using the following matrix $A$ and vector $b$:

In [ ]:
A = [2 0 4 3; -4 5 -7 -10; 1 15 2 -4.5; -2 0 2 -13];
b = [4,9,9,4];

First we calculate the LU factorization of $A$.

In [ ]:
L,U = lufact(A)

Now we can solve the linear equation by solving $Ly = b$ and $Ux = y$.

In [ ]:
z = forwardsub(L,b)
x = backsub(U,z)

And let's check the error of the solution:

In [ ]:
b - A*x

Nice! We actually found a way to solve linear equations in an efficient way. There is one caveat though. This method is not always stable and can fail for some matrices. For example if we swap the second and fourth row of $A$ the resluting matrix $A$ is not invertible and the LU factorization fails: 

In [ ]:
A[[2,4],:] = A[[4,2],:]  
L,U = lufact(A)
L

We can actually see why this fails if we take a look at the matrix $A$. The problem is that element $a_{22}$ is zero. In the second step we are supposed to divide by $a_{22}$ which is zero. 

In [ ]:
A

However, we know that this linear equation is actually solvable. Can we fix this? 

## Pivoting
To avoid dividing by zero, we can change the order of the coulmns before we perform the elimination in each column. We will use the largest available element in the column we are working on as the pivot element - the element we divide by. This technique is known as column pivoting. 

In [ ]:
A = [2 0 4 3 ; -2 0 2 -13; 1 15 2 -4.5 ; -4 5 -7 -10]

Using the same matirx as before the first step looks like this:

In [ ]:
A_1 = float(copy(A))
p = fill(0,4)
i = argmax(abs.(A_1[:,1])) # find largest element 
p[1] = i # store index of largest element in p[1]
L,U = zeros(4,4),zeros(4,4)
U[1,:] = A_1[i,:] # copy row i of A to row 1 of U 
# perform elimination as before
L[:,1] = A_1[:,1]/U[1,1] 
A_2 = A_1 - L[:,1]*U[1,:]'

The second step looks like this:

In [ ]:
i = argmax(abs.(A_2[:,2])) 
p[2] = i 
U[2,:] = A_2[i,:]
L[:,2] = A_2[:,2]/U[2,2]
A_3 = A_2 - L[:,2]*U[2,:]'

The third step looks like this:

In [ ]:
i = argmax(abs.(A_3[:,3])) 
p[3] = i
U[3,:] = A_3[i,:]
L[:,3] = A_3[:,3]/U[3,3]
A_4 = A_3 - L[:,3]*U[3,:]'

And the last step looks like this:

In [ ]:
i = argmax(abs.(A_4[:,4])) 
p[4] = i
U[4,:] = A_4[i,:]
L[:,4] = A_4[:,4]/U[4,4];
L

We can see that we can solve the linear equation by swapping the columns. However we can see that we also changed the order of the elements in the lower triangular matrix $L$. We can fix this by keeping track of the order of the columns and swapping the elements in the lower triangular matrix $L$ back accordingly: 

In [ ]:
L[p,:]

We can now write a function that calculates the LU factorization of a square matrix $A$ using column pivoting. The function should take the matrix $A$ as input and return the lower triangular matrix $L$, the upper triangular matrix $U$ and a vector $p$ containing the pivot elements.

In [ ]:
function plufact(A)
    n = size(A,1)
    L = zeros(n,n)
    U = zeros(n,n)
    p = fill(0,n)
    Aₖ = float(copy(A))

    # Reduction by outer products
    for k in 1:n-1
        p[k] = argmax(abs.(Aₖ[:,k])) # find largest element in column k and store in p[k]
        U[k,:] = Aₖ[p[k],:]
        L[:,k] = Aₖ[:,k]/U[k,k]
        Aₖ -= L[:,k]*U[k,:]'
    end
    p[n] = argmax(abs.(Aₖ[:,n]))
    U[n,n] = Aₖ[p[n],n]
    L[:,n] = Aₖ[:,n]/U[n,n]
    return LowerTriangular(L[p,:]),U,p
end

In [ ]:
L, U, p = plufact(A)

To solve for a given vector $b$ we can now use the pivot LU factorization of $A$ by also changing the order of the elements in the vector $b$ accordingly:

In [ ]:
b = rand(4)
z = forwardsub(L,b[p])
x = backsub(U,z)


In [ ]:
b - A*x

# Linear Equations in Julia 
there are different ways to solve a linear equation Julia. For basic linear equations we can use the backslash operator `\`. For example we can solve the linear equation $Ax = b$ by writing `x = A \ b`.

In [ ]:
A = [2 0 4 3 ; -2 0 2 -13; 1 15 2 -4.5 ; -4 5 -7 -10]
b = [4,9,9,4]
x = A\b

We can also do the LU factorization of $A$ using the `lu` function (from the LinearAlgebra package). The function returns the lower triangular matrix $L$, the upper triangular matrix $U$. 

In [ ]:
lu(A)

For more advanced linear equation solving we can use iterativesolvers.jl. It also contains the "Jacobi" and "Gauss-Seidel" solvers we had in the lecture. To get a list of solvers you can look at the documentation: https://iterativesolvers.julialinearalgebra.org/dev/ 